In [1]:
from pathlib import Path
import sys

ROOT = Path.cwd().resolve().parent
sys.path.insert(0, str(ROOT))

print("Repo root on sys.path:", ROOT)

Repo root on sys.path: /Users/ayaanmayooq/Desktop/code/greener_side


In [2]:
from utils.config import CFG, DATA_ROOT
print("DATA_ROOT =", DATA_ROOT)
print("Symbols:", CFG["symbols"]["swing"])

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'

In [ ]:
from utils.config import DATA_ROOT, CFG
import pandas as pd
from research.features.registry import build_features
from backtest.strategies.prob_triple_barrier import ProbTripleBarrier

sym = CFG["symbols"]["swing"][0]
bars = pd.read_parquet(DATA_ROOT / f"processed/daily/symbol={sym}/bars.parquet").sort_index()

strat = ProbTripleBarrier(model_id="swing_xgb_prob_tp_v1",
                          G_mult=3.0, L_mult=1.0, horizon=10,
                          margin=0.05, cost_bps=2.0)

fe = build_features(sym, bars, strat.required_feature_keys(), DATA_ROOT)

# Threshold math
C = strat.cost_bps()/10000.0
p_be = (strat.L_mult + C) / (strat.L_mult + strat.G_mult)
p_thr = p_be + strat.margin

stats = {
    "rows": len(fe),
    "pred_rows": int(fe["p_tp"].notna().sum()),
    "p_thr": float(p_thr),
    ">thr": int((fe["p_tp"] > p_thr).sum()),
    ">thr & 200dma": int(((fe["p_tp"] > p_thr) & (fe["adj_close"] > fe["sma_200"])).sum()),
    "p_mean": float(fe["p_tp"].mean()),
    "p_90": float(fe["p_tp"].quantile(0.90)),
    "p_95": float(fe["p_tp"].quantile(0.95)),
    "pred_start": fe["p_tp"].first_valid_index(),
    "pred_end": fe["p_tp"].last_valid_index(),
}
stats